In [1]:
import os
from openai import AzureOpenAI

In [2]:
from PyPDF2 import PdfReader

In [2]:
AZURE_OPENAI_PREVIEW_API_VERSION=os.environ.get("AZURE_OPENAI_PREVIEW_API_VERSION")
AZURE_OPENAI_API_KEY=os.environ.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT=os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_MODEL=os.environ.get("AZURE_OPENAI_MODEL")

In [3]:
client = AzureOpenAI(
    api_version= AZURE_OPENAI_PREVIEW_API_VERSION,
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

In [ ]:
reader = PdfReader('./OneDrive_1_10-4-2024/JobThai - 1032276_ฐาณภูมิ์_ธีรสีหรัตน์ - Application.pdf')
full_text = ""
for page in reader.pages:
    text = page.extract_text()
    full_text += text
print(full_text)

In [13]:
completion = client.chat.completions.create(
    model=AZURE_OPENAI_MODEL,
    messages=[
        {
            "role": "system",
            "content": """You are HR department assistant you job is to help summarize the data from job applicant
            the data you need to extract is the keyword of Job title and Skill only answer with the keyword
            """
        },
        {
            "role": "user",
            "content": full_text,
        },
    ],
)

In [ ]:
print(completion.to_json())

### Create search index

In [ ]:
! pip install azure-search-documents==11.6.0b1 --quiet
! pip install azure-identity --quiet
! pip install python-dotenv --quiet
! pip install azure-core

In [11]:
from azure.core.credentials import AzureKeyCredential
import os
from dotenv import load_dotenv

load_dotenv()

AZURE_SEARCH_API_KEY = os.environ.get("AZURE_SEARCH_API_KEY")
AZURE_SEARCH_API_ENDPOINT = os.environ.get("AZURE_SEARCH_API_ENDPOINT")

credential = AzureKeyCredential(AZURE_SEARCH_API_KEY)
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex
)


In [15]:
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_API_ENDPOINT, credential=credential)
fields = [
        SimpleField(name="resumeId", type=SearchFieldDataType.String, key=True),
        SimpleField(name="Name", type=SearchFieldDataType.String, sortable=True),
        SimpleField(name="PID", type=SearchFieldDataType.String, analyzer_name="th.lucene"),
        SearchableField(name="Intro", type=SearchFieldDataType.String, analyzer_name="th.lucene"),
        ComplexField(name="Work", collection=True, fields=[
            SearchableField(name="Title", type=SearchFieldDataType.String, analyzer_name="th.lucene"),
            SimpleField(name="Company", type=SearchFieldDataType.String),
            SimpleField(name="Date", collection=True, type=SearchFieldDataType.String),
            SearchableField(name="Duration", type=SearchFieldDataType.String, analyzer_name="th.lucene"),
            SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="th.lucene")
        ]),
        SearchableField(name="Education", collection=True,fields=[
            SearchableField(name="Degree", type=SearchFieldDataType.String, analyzer_name="th.lucene"),
            SimpleField(name="School", type=SearchFieldDataType.String),
            SimpleField(name="Date", collection=True, type=SearchFieldDataType.String),
            SearchableField(name="Duration", type=SearchFieldDataType.String, analyzer_name="th.lucene"),
        ]),
        SearchableField(name="Skill", collection=True, type=SearchFieldDataType.String, analyzer_name="th.lucene"),
        SearchableField(name="Certification", collection=True, type=SearchFieldDataType.String, analyzer_name="th.lucene"),
        SearchableField(name="Award", collection=True, type=SearchFieldDataType.String, analyzer_name="th.lucene"),
        SearchableField(name="Language", collection=True, type=SearchFieldDataType.String, filterable=True),
    ]
scoring_profiles = []
suggester = [{'name': 'sg', 'source_fields': ['Work/Description', 'Work/Duration', 'Skill', 'Certification', 'Language']}]


In [ ]:
index = SearchIndex(name="resume_index", fields=fields, suggesters=suggester, scoring_profiles=scoring_profiles)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

In [24]:
import json
import time

In [26]:
result_json_string = '''{  
    "name": "Phankawee Chulakasian",  
    "pid": "063-421-6162 | ch.phankawee_st@tni.ac.th | github.com/Phanxv | linkedin.com/in/phankawee-chulakasian-0870a32b5/",  
    "intro": "I'm currently a fourth-year college student at Thai-Nichi Institute of Technology, pursuing a degree in Computer Engineering. With a keen interest in Digital logic design, Embedded system and Full-Stack website development and I’m a fast learner and highly adaptable individual, ready to face new challenges.",  
    "work": [  
        {  
            "title": "Full-Stack Developer (Co-operative Education)",  
            "company": "Metro Systems Corporation Plc.",  
            "date": "1 JUL 2024 - 31 OCT 2024",  
            "duration": "4 months",  
            "description": "Worked on RPA projects to automate business processes, while also developing and deploying full-stack applications mainly using React and Python on Microsoft Azure cloud service."  
        },  
        {  
            "title": "Teaching Assistant",  
            "company": "Thai-Nichi Institute of Technology",  
            "date": [  
                "CPE-212 Digital Logic Design Laboratory: JUL 2023 - AUG 2023",  
                "CPE-112 Computer Programming Laboratory: JUN 2022 - OCT 2022",  
                "ENG-131 Calculus 1: JUL 2022 - SEP 2022"  
            ],  
            "duration": "Total 10 months",  
            "description": "Assisted in various laboratory courses."  
        },  
        {  
            "title": "Laboratory Staff",  
            "company": "Computer Engineering Robotic and Technology: CERT at Thai-Nichi Institute of Technology",  
            "date": [  
                "Laboratory Vice President: 2022 - 2023",  
                "Laboratory Student: 2021 - 2022"  
            ],  
            "duration": "Total 2 years",  
            "description": "Contributed to laboratory activities and management."  
        }  
    ],  
    "education": {  
        "degree": "Computer Engineering Major",  
        "school": "Thai-Nichi Institute of Technology",  
        "dates": "JUN 2021 - PRESENT",  
        "expected_graduation": "2024",  
        "gpax": "3.95"  
    },  
    "skills": [  
        "Quartus II for FPGA digital logic design",  
        "C/C++ and Arduino for microcontroller and IOT programming on ESP8266, ESP32, STM32 and Arduino microcontroller",  
        "Python for Machine Learning and Web Server programming",  
        "HTML,CSS,JS and React.js for Website development",  
        "Oracle SQL and MySQL for SQL database management",  
        "MongoDB and InfluxDB for NoSQL database management",  
        "Java for Android application development",  
        "Microsoft Azure for on-cloud deployment"  
    ],  
    "certifications": "NO_DATA",  
    "projects": "NO_DATA",  
    "award": [  
        "J-education Special award, TNI Japanese Presentation Contest. 16 OCT 2023",  
        "1st runner up, Deep Learning project award CPE-497. 25 SEP 2023",  
        "Winner of TNI-NIT Line Tracking Robot competition. 24 MAR 2023",  
        "1st runner up, Tech to Innovation, Innovation to Startup. 13 JAN 2023",  
        "Honorable certificate of academic excellence. 7 JUL 2023"  
    ],  
    "languages": [  
        "Thai - Native",  
        "English - Fluent (TOEIC 900)",  
        "Japanese - Basic proficiency (JLPT N4)"  
    ]  
}  '''

In [27]:
result_json = json.loads(result_json_string)

In [ ]:
result_json['name']

In [ ]:
mapped_data = {  
    "@search.action": "upload",
    "resumeId": str(time.time()),  # Generate a unique ID  
    "Name": result_json.get("name"),  
    "PID": result_json.get("pid"),  
    "Intro": result_json.get("intro"),  
    "Work": [  
        {  
            "Title": work["title"],  
            "Company": work["company"],  
            "Date": work["date"],  
            "Duration": work["duration"],  
            "Description": work["description"]  
        }  
        for work in result_json.get("work", [])  
    ],  
    "Education": [  
        f"{result_json['education']['degree']} at {result_json['education']['school']} ({result_json['education']['dates']})"  
    ],  
    "Skill": result_json.get("skills", []),  
    "Certification": result_json.get("certifications", []),  
    "Award": result_json.get("award", []),  
    "Language": result_json.get("languages", [])  
}  
  
# Convert the mapped data to JSON string  
mapped_json_string = json.dumps(mapped_data, indent=4)  
print(mapped_json_string)  


In [37]:
documents = [
    {
    "@search.action": "upload",
    "resumeId": f"{time.time()}",
    "Name": f"{result_json['name']}",
    "PID": f"{result_json['pid']}",
    "Intro": f"{result_json['intro']}",
    "Education": f"{result_json['education']}",
    "Skill": f"{result_json['skills']}",
    "Certification": f"{result_json['certifications']}",
    "Award": f"{result_json['award']}",
    "Language": f"{result_json['languages']}",
    "Work": f"{', '.join(f'{list(item.keys())[0]}: {list(item.values())[0]}' for item in result_json['work'])}"
    }
]

In [ ]:
print(documents)

In [ ]:
search_client = SearchClient(endpoint=AZURE_SEARCH_API_ENDPOINT,
                      index_name='resume_index',
                      credential=credential)
try:
    result = search_client.upload_documents(documents=mapped_data)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

    index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_API_ENDPOINT, credential=credential)